# Get the basic info

Though the URL didn't change when we go to next page, by Network Trace Analysis we could see the Request URL that in json format. And we could go to every page when the page number added to the last of template URL.

In [ ]:
# get info from: http://hk.centanet.com/findproperty/zh-HK/Home/SearchResult?posttype=S&src=D
import urllib.request
import json
URL = 'http://hk.centanet.com/findproperty/BLL/Result_SearchHandler.ashx?url=http%3A%2F%2Fhk.centanet.com%2Ffindproperty%2Fzh-HK%2FHome%2FSearchResult%3Fposttype%3DS%26src%3DF%26minprice%3D%26maxprice%3D%26sortcolumn%3D%26sorttype%3D%26limit%3D-1%26currentpage%3D'

for i in range(1,15891):
    with urllib.request.urlopen(f'{URL}{i}') as url:
        data = json.loads(url.read().decode()).get('post')
    print(f'page{i}_len : ', len(data))
    text_file = open(f'page{i}.html', "w")
    text_file.write(data)
    text_file.close()

# Extract infomation from scraped pages

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
from html.parser import HTMLParser
import pandas as pd

base = 'page'
suffix = '.html'

postid = []
locations = []
buildings = []
prices = []
rooms = []
usage = [] #實用率


for i in range(1,15891):
        url = f'{base}{i}{suffix}'
        #print(url)
        f = open(url)
        mypages = BeautifulSoup(f, 'html.parser')
        #print(mypages)
        for infos in mypages.find_all('div', attrs={'class': 'SearchResult_Row'}):               
                postid.append(infos['postid']) 
                for info in infos.find_all('div', attrs = {'class':'ContentInfo_Left fLeft'}):                                             
                    location = info.find('p').text
                    locations.append(location)
                    buildings.append(info.find('div', attrs = {'class':'ContentInfo_HeaderWrapper'}).text.strip())
                    prices.append(info.find('div', attrs = {'class':'price_size'}).text.strip().replace(',',"").replace('\n\n\n',",").replace('\n',","))
                    rooms.append(info.find('p', attrs = {'class':'ContentInfo_DetailStr_Lf fLeft OneRow'}))
                    usage.append(info.find_all('p', attrs = {'class':'ContentInfo_DetailStr_Lf leftBorder fLeft OneRow'}))


pd.set_option('display.max_colwidth', -1)

data = pd.DataFrame({
        "ids": postid,
        "locations": locations, 
        "buildings": buildings, 
        "prices": prices,
        "rooms": rooms,
        "usage":usage
       
    })
print(data)

data.to_csv("pages.csv",encoding='utf_8_sig')


# Get the number of schools

Using Network Trace Analysis we could see the number of schools also in json format, but the trick part here is the url is gerenated by the buildings `code`, and up to now we only have the `postid`. So I use the postid to go to every page and find the code one by one. 

Another question is one building may have serverl different `code`. It is because Centaline Property may named the code for different scope.  

So while I link the postid and code first, then link the code and numberOfSchools, after that group the code and numberOfSchools by postid, then sort every little group and droped duplicates to get the infomation.

In [ ]:
import csv 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import urllib.request
import re

df = pd.read_csv('pages.csv')
postid = df['ids']

URL = 'http://hk.centanet.com/findproperty/zh-HK/Home/Detail/?ID={}&src=S'
urls = []
links = []
codeLink = []
codes = []


# 
for i in range(len(postid)): 
    urls.append(URL.format(postid[i])) 

links = []
codeLink = []
codes = []
postidRE = []
for url in urls:
    source = requests.get(url)
    soup = BeautifulSoup(source.content, features='html.parser')
    for nav in soup.find_all('div', attrs = {'class':'navigationBar fLeft'}):
        for link in nav.find_all('a'):
            if re.search('code=', link.get('href')):
                code = link.get('href')[re.search('code=', link.get('href')).end():]
                codes.append(code)  
                postidRE.append(re.search(r'ID=(.+)&', url).group(1) ) 


data = pd.DataFrame({
        "postidRE": postidRE,
        "code": codes      
    })
data
# we could observer the school type here: kindergarten: 1, pre-school:2, middle-school:3, international-school:4
schoolURL = 'http://hk.centanet.com/findproperty/BLL/Detail_SchoolHandler.ashx?schooltype={}&cestcode={}&currentpage=1'

schools1 = []
schools2 = []
schools3 = []
schools4 = []
for i in range(len(codes)):
    schools1.append(schoolURL.format('1',codes[i])) 
    schools2.append(schoolURL.format('2',codes[i]))
    schools3.append(schoolURL.format('3',codes[i]))
    schools4.append(schoolURL.format('4',codes[i]))

# repeat for school 1 - 4
school1Codes = []
totalNumber1 = []
for i in range(len(schools1)):
    with urllib.request.urlopen(schools1[i]) as url:  
#         print(url)
        data = json.loads(url.read().decode())
        page = data.get('total')
        #print(page)
        if page == '0':
            pass
        else:
            school1Codes.append(re.search(r'code=(\w+)', schools1[i]).group(1)) 
            totalNumber1.append(page)
          
    
    
    data1 = pd.DataFrame({
        "code": school1Codes,
        "totalNumber1": totalNumber1
       
    })
#merge the datatrame with postid and code, code and numberOfSchools together, groupby postid, sort within groups, drop duplicates. 

#repeat for school 1-4
result1 = pd.merge(data, data1, how='left',on = 'code')
result11 = result1.groupby(['postidRE'], sort = False).apply(lambda x: x.sort_values('totalNumber1', ascending = False))
result_1 = result11.drop_duplicates(subset='postidRE',keep='first')

df0 = df.rename(index=str, columns={"ids": "postidRE"})
df1 = pd.merge(df0, result_1[['postidRE','totalNumber1']], how='left', on = 'postidRE')


df4.to_csv('withSchools.csv', encoding='utf_8_sig')